This notebook is a refinement use case after two quarters of prototyping work on the sky viewer.  As a result it makes some more direct references to specific tools like Aladin Lite, instead of a generic "sky viewer".

# Sky viewer to Spectrum visualization for an extragalactic field

This use case demonstrates how a scientist user can go from a sky view of Roman observations to an interactive view of slitless spectra taken by Roman. 

Prof. Sanzang is an experienced astronomer who has long been on a career-long quest searching for a class of galaxies he calls Sutras, which are rare compact objects with unusual spectra predicted to be near much more easily identified galaxies known as Vultures.  He learns from a conference in Chang'an about how Roman will be targeting some extragalactic fields with many Vultures in them and realizes the Roman slitless spectra may be perfect for identifying Sutras. At the conference, Sanzang meets an experienced research scientist, Sun Wukong, who has been seeking to atone for a previous failure attempting to take over a collaboration doing slitless spectroscopy with JWST. Wukong, being very clever but also impulsive, decides the Roman Research Nexus is the way to do this even though no one in the team has any experience with it.

Wukong quickly figures out how to start up the RRN and get the Aladin Lite skyviewer pointed at the target field with all the Vultures (see the other sky viewer cases for a examples of how he might have learned this). Knowing that the only way to identify Sutras are via spectroscopic data, he wants to see all the objects in the field with extracted Roman slitless spectra, and wants to inspect a subset that might be a Sutra. There are two ways Wukong might want to do this, so below we consider both cases: the "UI-based" approach and the "Astroquery approach".




## Case A: UI-based Spectral Search

In this case, Wukong works entirely with interactive visualization tools to find the spectrum he wants to examine.

He first needs a way to show the objects that the roman pipeline extracted as having spectra. There are several equally-valid ways this could work, but the most straightforward way is for there to be a toggle in the default RRN instance of Aladin Lite that just turns on marking all the extracted locations.  These would then appear as circles marking the relevant sources locations based on where the 0th order flux should be.  Using that, Wukong identifies a source that on the Aladin Lite sky view looks like it could be a Sutra and also has a spectrum.  Clicking on that then brings up a tooltip, side panel, or similar UI element that shows some details about that source, akin to the same information that the MAST portal would show about that specific target. Wukong would have a quick glance at that information to identify whether the object has the right properties to be a Sutra candidate.  He has to click on several objects before finding one that looks right, but once he finds it, he clicks on the "launch in Specviz" button that appears on the UI panel.  That triggers a new panel to appear in the RRN web interface that shows specviz pre-loaded with the specific target (analogous to what currently happens when you click a jdaviz-tool icon in the MAST portal).

Once the specviz view has appeared, Wukong quickly examines the spectrum and decides if it looks like the spectrum is roughly right for a Sutra. He'll have to look at several spectra to see a good one, so it should be easy to click on a new target and have specviz be update to show the new spectrum.  But once a good spectrum is found, Wukong clicks on a button something like "open in notebook", which opens up a Jupyter notebook that has as the first couple of cells exactly what is needed to launch specviz on the view Wukong has already been looking at.  This allows Wukong to continue with the workflow in the section below "The cases recombine".

## Case B: Astroquery-based Spectral Search

* create an overlay highlighting spectroscopic targets
* search for the targets in astroquer
* choose one programaticallly, launch specviz from there

Is this case, Wukong instead uses a code-based mechanism to do the search.  This approach is related to ["Sky Viewer Science Case: Do catalog searches in the platform, show them on the sky in Roman, do data-mining tasks"](sky-viewer-catalog-search.md), so some of the details there might also be relevant.

For this case, Wukong is starting from an Aladin Lite sky viewer window. To do this case he needs to get a Jupyter Notebook that is connected to that sky viewer.  Two possibilities for this first step (option 1 preferred, although it might make sense to start with 2 as a road to 1) are:

1. He clicks a button somewhere in the sky viewer that says "make a notebook from this view", which generates a notebook with just a few cells that ends with a sky viewer open in the RRN Jupyter Lab view that looks just like what he's looking at in the non-notebook UI.  I.e., it automatically centers on and zooms to the same state it's in in his starting view, with the relevant base image selected and shown.
2. There's some sort of "copy and paste this code into your notebook" which does the same as above, but he has to create his own notebook and paste it into the first cell.

Either way the result is a notebook that launches Aladin Lite in the RRN Jupyterlab view, with the Vulture-heavy field showing.  Wukong then executes something like the following:

In [ ]:
from astroquery.mast import Catalogs

catalog = Catalog.query_criteria(skyviewer.get_center(), radius=skyviewer.get_radius(), 
                    catalog='roman-sources')

`catalog` should now be an Astropy table that has all the roman sources within the sky viewer window Wukong was centered on.  This then allows Wukong to do some filtering in code, for example, picking out objects in a certain color-magnitude diagram box that are extended sources (i.e. not stars) by doing something like:

In [ ]:
extended_msk = catalog['source_type'] == 'extended'
mag_msk = (19 < catalog['F106_mag']) & (catalog['F106_mag'] < 20)

color = catalog['F0626_mag'] - catalog['F106_mag']
color_msk = (1 < color ) & (color < 2)

sutra_candidates = catalog[extended_msk & mag_msk & color_msk]

(Note the exact names of fields here are not necessarily that important - the point is that the filtering is on *some* kinds of scientifically-relevant columns that are in the catalogs)

Of course there may be several of these candidates that do *not* have spectra, so there also needs to be a way to determine which have actual spectra.  The code below is what Wukong would want to use, although if he instead has to do another `astroquery` call to match against some other catalog of spectroscopic objects, that's OK as long as it's clearly documented how to do that.

In [ ]:
sutra_spectroscopic_candidates = sutra_candidates[sutra_candidates['has_spectrum'] == True]

Lastly, Wukong wants to examine where those candidates are in the sky-viewer:

In [ ]:
skyviewer.add_marks(sutra_spectroscopic_candidates['coordinates'])

# OR - if it's not possible for query_criteria to return mixin SkyCoord columns:
skyviewer.add_marks(astropy.coordinates.SkyCoord(sutra_spectroscopic_candidates['ra'], sutra_spectroscopic_candidates['dec'], unit='deg'))

Now markers appear at all of the Sutras candidates Wukong identified, and Wukong zooms around the sky viewer examining them. He sees one that looks promising, and clicks on it. What pops up includes something that says `table_index = 12` (which of course would only make sense in the context of this particular notebook).  So then Wukong can do:

In [ ]:
best_candidate = sutra_spectroscopic_candidates[12]  # 12 comes from the UI interaction described above

from astroquery.mast import Observations
spectra = Observations.get_product_list(best_candidate['obsid'])
spectra = Observations.filter_products(spectra, productType='spectrum')

Now to actually get the spectrum, Wukong could potentially download it the normal `astroquery.mast` way:

In [ ]:
spectra_downloaded = Observations.download_products(spectra)

from jdaviz import Specviz

specviz = Specviz()
specviz.load_data(spectra_downloaded['Local Path'][0])
specviz.show()

OR (better but requires more work), the above cell could use some clever that either gets a direct file handle inside AWS if the notebook is run on the RRN or does the same thing as above if it's outside AWS, along the lines of:

In [ ]:
from jdaviz import Specviz

specviz = Specviz()
specviz.load_data(Observations.stream_product(spectra))
specviz.show()

Either way, Wukong now has a jdaviz window with specviz open on the specific preferred Sutra spectrum inside a notebook.

## The cases recombine

Whether choosing case A or case B, the flow from here is the same.  Wukong has a specviz view of the most likely Sutra and now needs to use spectral visualization tools to examine the selected spectrum.

Wukong first visually examines the spectrum by zooming in on various features. (He got good at examining spectra after being made to spend spending 500 days in a windowless office that was known as "Stone Mountain", for some reason...) Eventually he realizes this spectrum is indeed a good example of something like what theory predicts a Sutra should look like. But to be sure, he needs to determine how far away it is by getting a distance from the redshift of a particular emission line, which allows him to compute the amount of energy coming from that emission line.  First he needs to pull the spectrum out of Specviz into a notebook, which should be very straightforward (although it might look a bit different from this in upcoming versions of Jdaviz):

In [ ]:
spectrum = specviz.get_spectrum()  # returns an astropy Spectrum1D object

While he could use specviz itself to get an estimate of the centroid, he wants to use his own custom centroiding function that he likes better.  So he selects a spectral range in specviz using the UI and then does something like this:

In [ ]:
from specutils.analysis import line_flux

emission_line_region = specviz.get_selected_region() # yields a specutils.SpectralRegion object

line_flux = line_flux(spectrum, region=emission_line_region)
centroid = wukongs_centroiding_function(spectrum, region=emission_line_region)

redshift = (centroid - rest_wavelength) / rest_wavelength  # rest_wavelength is something Wukong knows from theory

But what Wukong actually needs is the *distance* not the redshift itself, so he can do:

In [ ]:
from astropy.cosmology import Planck18 as cosmo

distance = cosmo.luminosity_distance(redshift)

absolute_line_flux = line_flux * distance**-2

absolute_line_flux

Seeing the above number, he sees it's exactly consistant with theory.  He has found the Sutras! He tells Prof. Sanzang immediately, and they excitedly declare that their Journey to the West has been successful! (The Vulture field is in a field that is far to the west in RA.)

They write a paper and share it at the next conference in Chang'an, which absolves Wukong of the guilt of his previous bad behavior.  On their way out of the conference they see ascending into something called a Buddha tree, and have the stragest feeling it's related to their fate...

## Stretch Goals

The above is a minimal complete use case.  But extending from there are several additional directions that provide significant value for use cases similar to this one.  From here on should be considered "stretch goals" in that they are not absolutely necessary for the science case, but are extensions that the astronomy community following use cases like the above would likely benefit from.

Note that these are not in a particular priority order, as they may depend on feasibility of implementation for various pieces.

### Stretch Goal 1: Identify a source that's not in the list by clicking on a spot in the aladin lite view

This is essentially a mix of case A and case B above:  After starting Case B, Wukong realizes there's another source that was not in the programatically-defined criteria.  So Wukong decides to add the source he can see in the UI the same way as in Case A, but wants to add it to the `sutras_catalog` after the first cell in Case B.  This should be possible with something as straightforward as clicking on the source in Aladin Lite and then doing:

In [ ]:
target_location = sky_viewer.get_click() # this could be a bit more complicated, but should be ~ one line

catalog = Catalog.query_criteria(target_location, radius='closest row', 
                    catalog='roman-sources')  # this doesn't need to be taken literally, it could be a separate method or something, but should be approximately this simple

sutra_candidates = astropy.table.vstack(sutra_candidates, catalog)

### Stretch Goal 2: Spot a source *not* in the catalogs and examine it

In this scenario, instead of being like Case A or B, Wukong instead sees something in the image that is potentially a candidate Sutra, but for some reason was *not* recognized as an object by the standard Roman spectroscopic pipeline. So Wukong wants to examine the spectrum directly.  Wukong starts by clicking on the sources in Aladin Lite and grabbing the relevant direct images and spectroscopic data:

In [ ]:
target_location = sky_viewer.get_click() # same as above, this is not literal code but approximately this simple

# I don't know the relevant CAOM keywords as applied to Roman, but presumably there's *some* way to say "the image right here" and "the spectrum right here":
images = Observations.query_criteria(coordinates=target_location, radius=1*u.arcmin, somecaomkeyword='roman image')
spec2ds = Observations.query_criteria(coordinates=target_location, radius=1*u.arcmin, somecaomkeyword='roman slitless spectrum image')

image_files = Observations.download_data(Observations.get_product_list(images['obsid']))
spec2d_files = Observations.download_data(Observations.get_product_list(spec2ds['obsid']))

Wukong is already familiar with slitless spectra, so much so he has 72 different transformations he can use to extract spectra.  So he now runs his own custom code on the dataset:

In [ ]:
import wukong_spectrum_extraction

spectrum = wukong_spectrum_extraction.extract_spectrum(target_location, spec2d_files)

specviz.load_data(spectrum)

And now Wukong sees the spectrum is an even *better* Sutra that the pipeline missed.

### Stretch Goal 3: Run the actual pipeline on the target

This follows from stretch goal 2 but is sufficiently more complex that it should be thought of as an additional stretch goal.  Conceptually it's very simple.  Instead of the last cell above, the goal would be to be able to simply do

In [ ]:
import romancal

romancal.run_pipeline(image_files, spec2d_files, some_magic_keyword=target_location)

And have the pipeline run on that one target. This probably can't be done directly, as the SSC pipeline isn't publicly runnable like this.  But it might be possible to instead have it trigger some kind of async reprocessing.

### Stretch Goal 4: Repetition functionality for ~hundreds of sources

After completing the first paper, Sanzig and Wukong realize they may want to investigate several other candidates as they may turn out also to be Sutras. To do this, they follow the "case B" workflow above, up to the cell that begins with ``best_candidate = sutra_spectroscopic_candidates[12]``.  Instead, they should be able to do something like:

In [ ]:
many_spectra = Observations.get_product_list(sutra_spectroscopic_candidates['obsid'])
many_spectra = Observations.filter_products(many_spectra, productType='spectrum')
many_spectra_downloaded = Observations.download_products(many_spectra)

from jdaviz import Mosviz

mosviz = Mosviz()
mosviz.load_data(many_spectra_downloaded['Local Path'])
mosviz.show()

That then launches an instance of MOSViz that lets Wukong view the spectra together but still have most of the specviz functionality.